# Results

## Table of Contents
1. [Definint MonteCarloOptionPricing Class](#Introduction)
2. [Producing *American put prices* similar to *Simulated American* in table 1 in Longstaff.
 (to ensure correct implmenetation)-](#Data-Loading-and-Preparation)
3. [Finite Difference results (benchmark for analysis)](#Exploratory-Data-Analysis)

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os

# local
from LSM import MonteCarloOptionPricing
from FD import AmericanPutFiniteDifference

## 2.1 Producing *American put prices* similar to *Simulated American* in table 1 in Longstaff.
- Where the simulation is based on 100000 (50000 plus 50000 antithetic) paths for the stock price.